In [8]:
import cvxpy as cp
import numpy as np
import math

# Input data

## Front of house availability:
### Evening availability:
FH_evening_avail = [
    [1, 0, 1, 0, 1],
    [1, 1, 0, 1, 1],
    [1, 1, 0, 1, 1],
    [0, 1, 1, 1, 0],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 0, 0],
    [1, 1, 1, 1, 1],
    [0, 1, 0, 1, 0],
    [1, 0, 1, 1, 1],
    [1, 1, 1, 1, 1],
]

### Afternoon availability:

FH_afternoon_avail = [
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [0, 1, 1, 0, 1],
    [1, 0, 1, 1, 1],
    [0, 1, 1, 0, 1],
    [1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 1, 1, 0],
    [1, 0, 1, 1, 1],
]


## Back of house availability:
### Evening availability:
BH_evening_avail = [
    [1, 0, 1, 0, 1],
    [1, 1, 0, 1, 1],
    [1, 1, 0, 1, 1],
    [0, 1, 1, 1, 0],
    [1, 1, 1, 0, 0],
    [0, 0, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [0, 1, 0, 1, 0],
    [0, 1, 1, 1, 0],
    [1, 1, 1, 1, 1],
]

### Afternoon availability:
BH_afternoon_avail = [
    [1, 1, 1, 1, 1],
    [1, 1, 0, 1, 1],
    [0, 1, 1, 0, 1],
    [1, 0, 1, 1, 1],
    [1, 1, 1, 0, 0],
    [1, 0, 1, 1, 0],
    [0, 0, 0, 0, 0],
    [1, 1, 0, 1, 0],
    [0, 1, 1, 1, 0],
    [1, 0, 1, 1, 1],
]

## Wages
### Front of house afternoon and evening wages
wage_AFH = [100, 90, 75, 125, 60, 65, 80, 75, 95, 110]
wage_EFH = [105, 95, 80, 130, 65, 70, 85, 80, 100, 115]

### Back of house afternoon and evening wages
wage_ABH = [110, 65, 70, 90, 100, 65, 75, 85, 105, 100]
wage_EBH = [115, 70, 75, 95, 105, 70, 80, 90, 110, 105]

## Customer demand
demand = [100, 170, 120, 140, 120, 140, 150, 250, 150, 250]

# Model building & optimatization

## Decision variables
AFH = cp.Variable((10,5), boolean = True)
EFH = cp.Variable((10,5), boolean = True)
ABH = cp.Variable((10,5), boolean = True)
EBH = cp.Variable((10,5), boolean = True)

obj_func = cp.Minimize(
    cp.sum(cp.multiply(cp.reshape(wage_AFH, (10, 1)), AFH)) +
    cp.sum(cp.multiply(cp.reshape(wage_EFH, (10, 1)), EFH)) +
    cp.sum(cp.multiply(cp.reshape(wage_ABH, (10, 1)), ABH)) +
    cp.sum(cp.multiply(cp.reshape(wage_EBH, (10, 1)), EBH))
)


## Constraints
constraints = []

### Labor demand
for i in range(len(demand)):
    # Round up to nearest integer to determine number of employees required to meet demand.
    num_FOH = math.ceil(demand[i] / 50)
    num_BOH = math.ceil(demand[i] / 45)

    # If statement determines if demand is afternoon or evening
    if (i % 2 == 0):
        # Afternoon shifts
        constraints.append(cp.sum(AFH[:,i // 2]) >= num_FOH)
        constraints.append(cp.sum(ABH[:,i // 2]) >= num_BOH)
    else:
        # Evening shifts
        constraints.append(cp.sum(EFH[:,i // 2]) >= num_FOH)
        constraints.append(cp.sum(EBH[:,i // 2]) >= num_BOH)

### Shifts per employee
for i in range(AFH.shape[0]):
    constraints.append(cp.sum(ABH[i, :]) + cp.sum(EBH[i, :]) <= 8)
    constraints.append(cp.sum(ABH[i, :]) + cp.sum(EBH[i, :]) >= 1)
    constraints.append(cp.sum(AFH[i, :]) + cp.sum(EFH[i,:]) <= 8)
    constraints.append(cp.sum(AFH[i, :]) + cp.sum(EFH[i,:]) >= 1)
    
# Constraints for availability
for m in range(10):  # Loop over employees
    for n in range(5):  # Loop over days
        # Front of House constraints
        constraints.append(AFH[m, n] <= FH_afternoon_avail[m][n])
        constraints.append(EFH[m, n] <= FH_evening_avail[m][n])
        # Back of House constraints
        constraints.append(ABH[m, n] <= BH_afternoon_avail[m][n])
        constraints.append(EBH[m, n] <= BH_evening_avail[m][n])


problem = cp.Problem(obj_func, constraints)

#problem.solve(solver=cp.CVXOPT,verbose = True) #verbose parameter determines showing/not showing the output
problem.solve(solver=cp.GUROBI,verbose = True)

print("obj_func =")
print(obj_func.value)
print("AFH =")
print(np.round(AFH.value).astype(int))
print("EFH =")
print(np.round(EFH.value).astype(int))
print("ABH =")
print(np.round(ABH.value).astype(int))
print("EBH =")
print(np.round(EBH.value).astype(int))


                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Nov 24 04:45:56 PM: Your problem has 200 variables, 260 constraints, and 0 parameters.
(CVXPY) Nov 24 04:45:56 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 24 04:45:56 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 24 04:45:56 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 24 04:45:56 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 24 04:45:56 PM: Compiling problem (target solver=GUROBI).